读取数据集，清洗，生成提示词

In [1]:
import json
with open('new_dataset/1201train.json','r') as f:
    train_data = json.load(f)

In [7]:
print(train_data[2]['formal_statement'])
print(train_data[2]['formal_proof'][0])

theorem mathd_algebra_13 (a b : ℝ)
  (h₀ : ∀ x, x - 3 ≠ 0 ∧ x - 5 ≠ 0 → 4 * x / (x ^ 2 - 8 * x + 15) = a / (x - 3) + b / (x - 5)) :
  a = -6 ∧ b = 10 := by

  /-
  To find \( A \) and \( B \) such that
  \[
  \frac{4x}{x^2 - 8x + 15} = \frac{A}{x-3} + \frac{B}{x-5}
  \]
  for all \( x \) besides 3 and 5, we need to solve for \( A \) and \( B \). We will use the given equation and substitute specific values of \( x \) to form a system of equations. By solving this system, we will determine the values of \( A \) and \( B \).
  -/
  have : a = -6 ∧ b = 10 := by
    -- Substitute specific values of x to form a system of equations
    have := h₀ 4
    have := h₀ 2
    have := h₀ 1
    have := h₀ (-1)
    have := h₀ (-2)
    have := h₀ (-3)
    have := h₀ (-4)
    have := h₀ (-5)
    have := h₀ (-6)
    have := h₀ (-7)
    have := h₀ (-8)
    have := h₀ (-9)
    have := h₀ (-10)
    -- Normalize the equations to solve for a and b
    norm_num at *
    -- Solve the system of equations to find

In [1]:
#以下是gpt构造器_用于生成提示词
from openai import AzureOpenAI
import json
clientT = AzureOpenAI(

    api_version="2023-05-15"
)

save_pth='new_dataset/'

class gpt_interface:
    def __init__(self):
        with open("/home/limengze/chen_dongheng/pipeline/new_dataset/history.json",'r') as f:
            self.history=json.load(f)
    def get_response(self,id,question):
        messages=[
            {
                "role": "system",
                "content": '''
You are a math assistant. Please give me a natural language proof for the following question in one line: 
'''
            },
            {
                "role": "user",
                "content": question
            }
        ]
        if question in self.history:
            return id,self.history[question]
        #不在，则调用gpt

        response = clientT.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
        result_str=(response.choices[0].message.content)
        self.history[question]=result_str
        return id,result_str
    
    def save(self):
      with open(save_pth+'history.json','w') as f:
          json.dump(self.history,f)
gpt = gpt_interface()

In [3]:
#以下是多线程构造器
import concurrent.futures
def parallel_requests(func,prompts_list,max_process=16):
    Flag = True
    results = [0]*len(prompts_list)
    # 使用线程池来并行处理请求
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_process) as executor:
        # 提交所有的请求到线程池
        futures = [executor.submit(func,id,item) for id,item in enumerate(prompts_list)]
        # 收集所有请求的结果
        for future in concurrent.futures.as_completed(futures):
            try:
                id,result = future.result()
                results[id]= result
            except Exception as exc:
                Flag = False
    return Flag,results

In [3]:
#测试一下生成
gpt.get_response(0,'Prove 2^n > n^2 for n>4')

(0,
 'For n > 4, observe that the base case holds at n = 5 with 2^5 = 32 > 25 = 5^2, and by induction, assume 2^k > k^2 for some k > 4, which implies 2^(k+1) = 2 * 2^k > 2 * k^2 > (k+1)^2 (since k > 4), hence 2^(k+1) > (k+1)^2, completing the proof.')

In [5]:
# 按顺序打包所有的题目
questions = [item['formal_statement'] for item in train_data]
# 并行生成所有的答案
success,prompts = parallel_requests(gpt.get_response,questions)

In [6]:
# 删除证明的注释
import re
def remove_proof_comments(proof):
    proof = re.sub(r'/-[\s\S]*?-\/', '', proof,re.DOTALL)
    # 删除 -- 开头的行
    lines = proof.splitlines()
    lines = [line for line in lines if not line.strip().startswith('--')]
    # 删除空行
    lines = [line for line in lines if line.strip()]
    # 重新合成行
    text =  '\n'.join(lines)
    # 删除/-开头，-/结尾的所有字符串
    
    return text



In [12]:
#构造训练集
train_set=[]
for i in range(len(prompts)):
    temp = {}
    temp['instruction'] = 'Complete the following Lean 4 code\n\n```lean4\n%s\n``` '%train_data[i]['formal_statement']+'\n--%s\n'%prompts[i]
    temp['output'] ='\n' + remove_proof_comments(train_data[i]['formal_proof'][0])
    train_set.append(temp)
    

In [13]:
with open('1201_train_pairs.json','w') as f:
    json.dump(train_set,f)